In [ ]:
# jupyter-notebook extensions
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

# custom import of built umpire package
from umpire import UMPIRE
# some plotting functions
from umpire.utils import plot_image_series

# custom manual import of test lib
import sys
sys.path.append("../tests")

from test_umpire import generate_simulated_data_2D, wrap_phase

### Define helpful noise function

In [ ]:
def noisy(noise_typ,image):
    """Noisyfy images.
    
    https://stackoverflow.com/questions/22937589/how-to-add-noise-gaussian-salt-and-pepper-etc-to-image-in-python-with-opencv
    
    Parameters
    ----------
    image : ndarray
        Input image data. Will be converted to float.
    mode : str
        One of the following strings, selecting the type of noise to add:

        'gauss'     Gaussian-distributed additive noise.
        's&p'       Replaces random pixels with 0 or 1.
        'speckle'   Multiplicative noise using out = image + n*image,where
                    n is uniform noise with specified mean & variance.
    """
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.5
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1
        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss * 0.02
        return noisy

In [ ]:
TEs = [5, 10, 16]
img_dims = (64, 64)

## Normal 2D simulated data

In [ ]:
phase_imgs = generate_simulated_data_2D(img_dims, TEs, reciever_offset=True)
# wrap phase
phase_imgs_wrapped = wrap_phase(phase_imgs)

plot_image_series(np.concatenate((phase_imgs, phase_imgs_wrapped)),
                  [f"TE = {t} ms\nphase" for t in TEs] + ["wrapped phase"]*3,
                  nrows=2,
                  figsize=(9,5))

## Noisy simulated data: 3rd echo with gaussian noise

In [ ]:
# Add noise to third echo image
phase_imgs_noisy = np.copy(phase_imgs)
phase_imgs_noisy[2] = np.squeeze(noisy("gauss", np.expand_dims(phase_imgs[2], axis=2)))
# wrap phase
phase_imgs_noisy_wrapped = wrap_phase(phase_imgs_noisy)

plot_image_series(np.concatenate((phase_imgs_noisy, phase_imgs_noisy_wrapped)),
                  [f"TE = {t} ms\n{x} phase" for t, x in zip(TEs, ["", "", "noisy"])] + ["wrapped phase"] * 2 + ["noisy wrapped phase"],
                  nrows=2,
                  figsize=(9,5))

## Ok now lets create some magnitude images

In [ ]:
magnitude_imgs = np.ones((3, *img_dims))
magnitude_imgs[2] *= 1

plot_image_series(magnitude_imgs, [f"TE = {t} ms" for t in TEs], nrows=1, normalize=True, figsize=(9,3))

## UMPIRE

First, combine phase and magnitude into complex array:

In [ ]:
complex_imgs = magnitude_imgs * np.exp(1j * phase_imgs)
complex_imgs_noisy = magnitude_imgs * np.exp(1j * phase_imgs_noisy)

### STEP 2 - Phase Difference Images

In [ ]:
out_2 = UMPIRE(
    complex_imgs_noisy,
    TEs,
    DPD_filter_func=None,
    magnitude_weighted_omega_star=True,
    debug_return_step=2
)
plot_image_series(out_2,
                  ["PD21", "PD32"],
                  nrows=1,
                  figsize=(9,3))

In [ ]:
out_6 = UMPIRE(
    complex_imgs_noisy,
    TEs,
    DPD_filter_func=None,
    magnitude_weighted_omega_star=True,
    debug_return_step=6
)
plot_image_series(out_6,
                  ["PD21_prime", "PD32_prime"],
                  nrows=1,
                  normalize=True,
                  figsize=(9,3))

### STEP 7 - $\omega^*$-image

In [ ]:
out_7 = UMPIRE(
    complex_imgs_noisy,
    TEs,
    DPD_filter_func="default",
    magnitude_weighted_omega_star=True,
    debug_return_step=7
)

plot_image_series(out_7,
                  [r"$\omega^*$ PD21 only", r"$\omega^*$ with mw-PDs"],
                  nrows=1,
                  normalize=True,
                  figsize=(9,5))

# plot_image_series(np.log(np.abs(phase_imgs_umpire)),
#                   ["PD21 only", "mw-avg. PDs"],
#                   nrows=1,
#                   normalize=True,
#                   figsize=(9,5))

In [ ]:
fig = plt.figure(figsize=(5,4))
plt.imshow(np.abs(out_7[1] - out_7[0]), cmap='Reds')
plt.colorbar()
plt.title("|PD21 - mw-avg. PDs|")
fig.tight_layout()

## UMPIRE final output

In [ ]:


# not magnitude weighted
phase_imgs_umpire = UMPIRE(
    complex_imgs_noisy,
    TEs,
    DPD_filter_func="default",
    magnitude_weighted_omega_star=False,
)

# magnitude weighted
phase_imgs_umpire_mw = UMPIRE(
    complex_imgs_noisy,
    TEs,
    DPD_filter_func="default",
    magnitude_weighted_omega_star=True,
)

## Comparison with ground truth

In [ ]:
grnd_truth = generate_simulated_data_2D(img_dims, TEs, reciever_offset=False)

abs_error = np.abs(grnd_truth - phase_imgs_umpire)
abs_error_mw = np.abs(grnd_truth - phase_imgs_umpire_mw)

print(f"no mw: {np.sum(abs_error):.0f}")
print(f"   mw: {np.sum(abs_error_mw):.0f}")

In [ ]:
plot_image_series(np.concatenate((abs_error, abs_error_mw)),
                  [f"TE = {t} ms\nabs. error mwOFF" for t in TEs] + ["abs. error mwON "] * 3,
                  nrows=2,
                  normalize=False,
                  cmap='Reds',
                  figsize=(9,7))